This notebook reads in the conbined SNe catalog file ('1.FullCatalog.csv') and searches through PHANGS CO map images, looking for OSC objects that have gone off since a chosen year. 


In [1]:
#import packages and formatting statements

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import ascii

plt.rc('text',usetex=False)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

In [2]:
# assign previous and future year boundaries for survey, list actual cutoff year for filename

PREVIOUS_YEAR = "2018"
FUTURE_YEAR = "2023"
CUTOFF_YEAR = "2019"

In [3]:
FullCat = '/home/maykerchen.1/Desktop/PHANGS_SNe/Data/1.FullCatalog.csv'
SNTable = Table.read(FullCat, format='csv')
SNTable

Name,Host,RA,Dec,Type,DiscDate
str41,str46,str11,str12,str19,str23
"""SN 2022yjl""","""""",22:17:58.32,+10:59:43.14,SN II,2022-10-22 15:08:59.136
"""SN 2022yig""","""""",5:20:21.49,-20:54:41.44,SN Ia,2022-10-21 06:59:41.280
"""SN 2022yid""","""NGC 946""",2:30:38.61,+42:13:57.65,SN Ia,2022-10-21 11:31:12.000
"""SN 2022yhl""","""""",2:27:30.97,-1:06:35.15,SN Ia,2022-10-21 11:46:22.656
"""SN 2022ydl""","""""",22:40:04.42,-6:38:28.21,SN Ia,2022-10-19 02:12:36.576
"""SN 2022ydi""","""""",1:08:00.62,+40:14:00.32,SN Ia,2022-10-19 09:53:19.392
"""SN 2022ycs""","""""",7:50:56.43,+18:37:35.35,SN II,2022-10-14 11:25:26.400
"""SN 2022ybo""","""""",10:32:16.23,+53:38:38.9,SN Ia,2022-10-15 11:09:36.000
"""SN 2022ybb""","""""",23:30:06.18,+5:40:37.28,SN Ia,2022-10-18 09:35:28.896


In [4]:
# shorten DiscDate column to only have year

DiscYear = [c[0:4] for c in SNTable["DiscDate"]]
SNTable.add_column(DiscYear, name="DiscYear")


In [5]:
# remove rows of SNTable that have year < year chosen

mask = (SNTable["DiscYear"] > PREVIOUS_YEAR) & (SNTable["DiscYear"] < FUTURE_YEAR) 
SNRecent = SNTable[mask] 

SNRecent


Name,Host,RA,Dec,Type,DiscDate,DiscYear
str41,str46,str11,str12,str19,str23,str4
"""SN 2022yjl""","""""",22:17:58.32,+10:59:43.14,SN II,2022-10-22 15:08:59.136,2022
"""SN 2022yig""","""""",5:20:21.49,-20:54:41.44,SN Ia,2022-10-21 06:59:41.280,2022
"""SN 2022yid""","""NGC 946""",2:30:38.61,+42:13:57.65,SN Ia,2022-10-21 11:31:12.000,2022
"""SN 2022yhl""","""""",2:27:30.97,-1:06:35.15,SN Ia,2022-10-21 11:46:22.656,2022
"""SN 2022ydl""","""""",22:40:04.42,-6:38:28.21,SN Ia,2022-10-19 02:12:36.576,2022
"""SN 2022ydi""","""""",1:08:00.62,+40:14:00.32,SN Ia,2022-10-19 09:53:19.392,2022
"""SN 2022ycs""","""""",7:50:56.43,+18:37:35.35,SN II,2022-10-14 11:25:26.400,2022
"""SN 2022ybo""","""""",10:32:16.23,+53:38:38.9,SN Ia,2022-10-15 11:09:36.000,2022
"""SN 2022ybb""","""""",23:30:06.18,+5:40:37.28,SN Ia,2022-10-18 09:35:28.896,2022


In [8]:
fileName = "../Data/2.RecentSNTable" + CUTOFF_YEAR + ".csv"
SNRecent.write(fileName, overwrite = True)
